In [1]:
#!/usr/bin/env python

In [2]:
# -*- coding: utf-8 -*-

In [3]:
import numpy as np

In [4]:
import pandas as pd

In [5]:
import tqdm

In [6]:
import torch

In [7]:
import torch.nn.functional as F

In [8]:
from sklearn.ensemble import RandomForestRegressor

In [9]:
from sklearn.linear_model import LinearRegression

In [10]:
from sklearn.metrics import r2_score as r2

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
from sklearn.preprocessing import MinMaxScaler

In [13]:
from sklearn.preprocessing import StandardScaler

In [14]:
from torch import nn

In [15]:
from torch import optim

In [16]:
from torch.optim import Optimizer

In [17]:
# import python_libs

In [18]:
np.random.seed(42)

In [19]:
# parameters model

In [20]:
torch.manual_seed(42)

In [21]:
# parameters torch model

In [22]:
df = pd.read_csv('recepts.csv')

In [23]:
# download recepts.csv

In [24]:
df.head()

,recipecategory,keywords_1,keywords_2,keywords_3,keywords_4,keywords_5,0,1,2,3,...,7,8,9,10,11,aggregatedrating,reviewcount,calories,fatcontent,saturatedfatcontent
0,26,2,9,3,6,10,2.0,1.0,1.0,4.0,...,2.0,1.0,1.0,4.0,4.0,4.5,4.0,170.9,2.5,1.3
1,15,52,12,4,1,10,3.0,2.0,2.0,1.0,...,3.0,2.0,2.0,1.0,1.0,3.0,1.0,1110.7,58.8,16.6
2,8,10,6,5,3,1,1.0,1.0,1.0,12.0,...,1.0,1.0,1.0,12.0,12.0,4.5,10.0,311.1,0.2,0.0
3,76,17,2,3,14,1,2.0,1.0,2.0,46.0,...,2.0,1.0,2.0,46.0,46.0,4.5,2.0,536.1,24.0,3.8
4,4,10,7,3,6,19,1.0,1.0,1.0,12.0,...,1.0,1.0,1.0,12.0,12.0,4.5,11.0,103.6,0.4,0.1


In [25]:
# show head() recepts.csv

In [26]:
df.isna().sum()

recipecategory         0
keywords_1             0
keywords_2             0
keywords_3             0
keywords_4             0
keywords_5             0
0                      0
1                      0
2                      0
3                      0
4                      0
5                      0
6                      0
7                      0
8                      0
9                      0
10                     0
11                     0
aggregatedrating       0
reviewcount            0
calories               0
fatcontent             0
saturatedfatcontent    0
dtype: int64

In [27]:
# show isna().sum() recepts.csv

In [28]:
print(df.dtypes)

recipecategory           int64
keywords_1               int64
keywords_2               int64
keywords_3               int64
keywords_4               int64
keywords_5               int64
0                      float64
1                      float64
2                      float64
3                      float64
4                      float64
5                      float64
6                      float64
7                      float64
8                      float64
9                      float64
10                     float64
11                     float64
aggregatedrating       float64
reviewcount            float64
calories               float64
fatcontent             float64
saturatedfatcontent    float64
dtype: object


In [29]:
# show dtypes() recepts.csv

In [30]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 522517 entries, 0 to 522516
Data columns (total 23 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   recipecategory       522517 non-null  int64  
 1   keywords_1           522517 non-null  int64  
 2   keywords_2           522517 non-null  int64  
 3   keywords_3           522517 non-null  int64  
 4   keywords_4           522517 non-null  int64  
 5   keywords_5           522517 non-null  int64  
 6   0                    522517 non-null  float64
 7   1                    522517 non-null  float64
 8   2                    522517 non-null  float64
 9   3                    522517 non-null  float64
 10  4                    522517 non-null  float64
 11  5                    522517 non-null  float64
 12  6                    522517 non-null  float64
 13  7                    522517 non-null  float64
 14  8                    522517 non-null  float64
 15  9                

In [31]:
# show info() recepts.csv

In [32]:
print(df.shape)

(522517, 23)


In [33]:
# show shape() recepts.csv

In [34]:
Y = df['saturatedfatcontent']

In [35]:
# show target(Y) recepts.csv

In [36]:
X = df.iloc[:, 0:22]
y = df.iloc[:, 22]

In [37]:
# show target(y) recepts.csv

In [38]:
X.head()

,recipecategory,keywords_1,keywords_2,keywords_3,keywords_4,keywords_5,0,1,2,3,...,6,7,8,9,10,11,aggregatedrating,reviewcount,calories,fatcontent
0,26,2,9,3,6,10,2.0,1.0,1.0,4.0,...,8.0,2.0,1.0,1.0,4.0,4.0,4.5,4.0,170.9,2.5
1,15,52,12,4,1,10,3.0,2.0,2.0,1.0,...,3.0,3.0,2.0,2.0,1.0,1.0,3.0,1.0,1110.7,58.8
2,8,10,6,5,3,1,1.0,1.0,1.0,12.0,...,2.0,1.0,1.0,1.0,12.0,12.0,4.5,10.0,311.1,0.2
3,76,17,2,3,14,1,2.0,1.0,2.0,46.0,...,1.0,2.0,1.0,2.0,46.0,46.0,4.5,2.0,536.1,24.0
4,4,10,7,3,6,19,1.0,1.0,1.0,12.0,...,1.0,1.0,1.0,1.0,12.0,12.0,4.5,11.0,103.6,0.4


In [39]:
# show head() recepts.csv

In [40]:
print(y[:5])

0     1.3
1    16.6
2     0.0
3     3.8
4     0.1
Name: saturatedfatcontent, dtype: float64


In [41]:
# show target(y) recepts.csv

In [42]:
print(X.describe())

       recipecategory     keywords_1     keywords_2     keywords_3  \
count   522517.000000  522517.000000  522517.000000  522517.000000   
mean        16.309261      16.171478      13.838792       9.577702   
std         22.785018      20.981579      17.691402      12.721010   
min          1.000000       1.000000       1.000000       1.000000   
25%          1.000000       4.000000       3.000000       2.000000   
50%          7.000000       8.000000       7.000000       5.000000   
75%         21.000000      19.000000      17.000000      10.000000   
max        208.000000     195.000000     167.000000     141.000000   

          keywords_4     keywords_5              0              1  \
count  522517.000000  522517.000000  522517.000000  522517.000000   
mean        9.005596       7.506607       6.531346       6.590226   
std        10.283178       7.904510      39.700560      41.644158   
min         1.000000       1.000000       0.250000       0.250000   
25%         2.000000    

In [43]:
# show describe() recepts.csv

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [45]:
# train = 0.7, test = 0.3

In [46]:
scaler = StandardScaler()

In [47]:
# SkLearn.StandardScaler()

In [48]:
X_train = scaler.fit_transform(X_train)

In [49]:
# transform X_train

In [50]:
X_test = scaler.transform(X_test)

In [51]:
# transform X_test

In [52]:
scaler = MinMaxScaler()

In [53]:
# SkLearn.MinMaxScaler()

In [54]:
regressor = RandomForestRegressor(n_estimators=100,
                                      criterion='poisson', max_depth=14, max_features=9, n_jobs = 8, random_state=40)

In [55]:
# n_estimators=100, criterion='poisson', max_depth=14, max_features=9, n_jobs = 8, random_state=40

In [56]:
regressor.fit(X_train, y_train)

RandomForestRegressor(criterion='poisson', max_depth=14, max_features=9,
                      n_jobs=8, random_state=40)

In [57]:
# SkLearn.RandomForestRegressor()

In [58]:
y_pred = regressor.predict(X_test)

In [59]:
# y_predict X_test

In [60]:
r2(y_test, y_pred)

0.8111187803541133

In [61]:
# SkLearn.R2_score

In [62]:
print(r2(y_test, y_pred))

0.8111187803541133


In [63]:
# SkLearn.R2_score

In [64]:
X_train_scaled = scaler.fit_transform(X_train)

In [65]:
# transform X_train

In [66]:
X_test_scaled = scaler.transform(X_test)

In [67]:
# transform X_test

In [68]:
X_train_tensor = torch.Tensor(np.array(X_train_scaled))

In [69]:
# transform X_train tensor

In [70]:
X_test_tensor = torch.Tensor(np.array(X_test_scaled))

In [71]:
# transform X_test tensor

In [72]:
y_train_tensor = torch.Tensor(np.array(y_train))

In [73]:
# transform y_train tensor

In [74]:
y_test_tensor = torch.Tensor(np.array(y_test))

In [75]:
# transform y_test tensor

In [76]:
n_data, n_features = X_train_tensor.shape

In [77]:
# show shape() n_data, n_features

In [78]:
print(n_data)

365761


In [79]:
# show shape() n_data

In [80]:
print(n_features)

22


In [81]:
# show shape() n_features

In [82]:
def loss(mean, target):
    """function loss"""
    return mean(F.l1_loss(input, target, reduction="none") / target) * 100

In [83]:
# """function loss"""

In [84]:
loss_func = F.mse_loss

In [85]:
# loss_func

In [86]:
metrics_func_1 = [loss_func, loss]

In [87]:
# loss()

In [88]:
metrics_name = ["MSE", "LOSS"]

In [89]:
# loss()

In [90]:
def print_metrics(models, train_data, test_data, models_name):
    """function print_metrics"""
    results = np.ones(2 * len(models), len(metrics_func_1))
    models_name = []
    for model in models_name:
        models_name.extend([model + "Train", model + "Test"])
    for row, sample in enumerate([train_data, test_data]):
        results[row + sample * 2] = evaluate(models, metrics_func_1, sample[0], sample[1])
        results = pd.DataFrame(results, columns=metrics_name, index=models_name)
        return results

In [91]:
# """function print_metrics"""

In [92]:
train_data_1 = (X_train_tensor, y_train_tensor)

In [93]:
# train_data_1

In [94]:
test_data_1 = (X_test_tensor, y_test_tensor)

In [95]:
# test_data_1

In [96]:
model_lr_sklearn = LinearRegression()

In [97]:
# SkLearn.LinearRegression()

In [98]:
model_lr_sklearn.fit(X_train_scaled, y_train)

LinearRegression()

In [99]:
# SkLearn.LinearRegression()

In [100]:
model_1 = [model_lr_sklearn.predict]

In [101]:
# model_lr_sklearn.predict

In [102]:
metrics_name = ["MSE", "LOSS"]

In [103]:
# loss()

In [104]:
models_name_1 = ["LOSS"]

In [105]:
# loss()

In [106]:
model_lr = nn.Sequential(
    nn.Linear(in_features=n_features,
    out_features=1))

In [107]:
# torch model_lr

In [108]:
print(model_lr)

Sequential(
  (0): Linear(in_features=22, out_features=1, bias=True)
)


In [109]:
# torch model_lr

In [110]:
metrics_name_1 = torch.nn.MSELoss()

In [111]:
# torch.nn.MSELoss()

In [112]:
Optimizer = optim.SGD(params=model_lr.parameters(), lr=0.001)

In [113]:
# torch.optim model_lr

In [114]:
BATCH_SIZE_LR = 16

In [115]:
# torch batch

In [116]:
EPOCHS_LR = 1000

In [117]:
# torch epochs

In [118]:
for epoch in tqdm.trange(EPOCHS_LR):
    for i in range((n_data - 1) // BATCH_SIZE_LR + 1):
        start_i = i * BATCH_SIZE_LR
        end_i = start_i + BATCH_SIZE_LR
        Xb = X_train_tensor[start_i: end_i]
        yb = y_train_tensor[start_i: end_i]
        pred = yb
        loss_1 = loss_func(pred, yb + 1)

100%|███████████████████████████████████████| 1000/1000 [06:23<00:00,  2.61it/s]


In [119]:
# for epoch in tqdm.trange(EPOCHS_LR)

In [120]:
print(X_test)

[[ 2.14267765  0.039095    0.79884574 ... -0.01879476 -0.09096936
  -0.05191591]
 [ 0.64735879 -0.48534494 -0.66910215 ... -0.20382646 -0.04115568
  -0.14984893]
 [-0.67204021  3.51946916 -0.21742588 ... -0.20382646 -0.20807853
  -0.18840524]
 ...
 [ 0.11959919 -0.67605037 -0.49972355 ... -0.01879476 -0.23577368
  -0.16835596]
 [-0.18826058 -0.48534494 -0.21742588 ... -0.06505269  0.20665959
   0.22106275]
 [ 0.16357915 -0.67605037 -0.49972355 ...  1.8777802  -0.25087444
  -0.15370457]]


In [121]:
# X_test

In [122]:
print(y_test)

360511     2.6
392362     1.4
441510     0.0
106802     0.1
53        12.1
          ... 
326281    16.3
64228      0.6
340331     0.9
1465      18.8
40621      1.1
Name: saturatedfatcontent, Length: 156756, dtype: float64


In [123]:
# y_test

In [124]:
X = torch.randn(BATCH_SIZE_LR, 22)

In [125]:
# torch.randn()

In [126]:
y = torch.Tensor([[1.0], [0.0], [0.0], [1.0], [1.0],
    [1.0], [0.0], [0.0], [1.0], [1.0],
    [1.0], [0.0], [0.0], [1.0], [1.0],
    [1.0]])

In [127]:
# torch tensor

In [128]:
for epoch in range(1000):
    y_pred = model_lr(X)
    loss = metrics_name_1(y_pred, y)
    print('epoch: ', epoch,' loss: ', loss.item())
    Optimizer.zero_grad()
    loss.backward()
    Optimizer.step()

epoch:  0  loss:  0.6822163462638855
epoch:  1  loss:  0.6770461201667786
epoch:  2  loss:  0.6719217300415039
epoch:  3  loss:  0.6668428182601929
epoch:  4  loss:  0.661808967590332
epoch:  5  loss:  0.6568197011947632
epoch:  6  loss:  0.6518747210502625
epoch:  7  loss:  0.6469733715057373
epoch:  8  loss:  0.6421152949333191
epoch:  9  loss:  0.6373001933097839
epoch:  10  loss:  0.6325275897979736
epoch:  11  loss:  0.6277970671653748
epoch:  12  loss:  0.6231082677841187
epoch:  13  loss:  0.6184607744216919
epoch:  14  loss:  0.6138541102409363
epoch:  15  loss:  0.6092879176139832
epoch:  16  loss:  0.6047619581222534
epoch:  17  loss:  0.6002756357192993
epoch:  18  loss:  0.5958287119865417
epoch:  19  loss:  0.5914207100868225
epoch:  20  loss:  0.5870513319969177
epoch:  21  loss:  0.5827202796936035
epoch:  22  loss:  0.5784269571304321
epoch:  23  loss:  0.574171245098114
epoch:  24  loss:  0.5699526071548462
epoch:  25  loss:  0.5657709240913391
epoch:  26  loss:  0.561

In [129]:
# for epoch in range(1000)

In [130]:
def train(model_lr, loss_fn, optimizer):
    """function train"""
    size = len(X_train_tensor)
    model_lr.train()
    for batch_size_lr in range(1000):
        y_pred_1 = model_lr(X)
        loss = loss_fn(y_pred_1, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if BATCH_SIZE_LR % 16 == 0:
            loss_2 = loss.item(), BATCH_SIZE_LR * len(X)
            print(f'loss: {loss_2}')

In [131]:
# """function test"""

In [132]:
def test(model_lr, loss_fn):
    """function test"""
    size = len(X_train_tensor)
    num_batches = size
    model_lr.eval()
    test_loss = 0
    with torch.no_grad():
        for batch_size_lr in range(1000):
            y_pred_2 = model_lr(X)
            test_loss += loss_fn(y_pred_2, y).item()
            test_loss /= num_batches
            print(f'Test loss: {test_loss}')

In [133]:
# """function test"""

In [134]:
loss_fn = nn.CrossEntropyLoss()

In [135]:
# loss_fn()

In [136]:
# for epoch in range(EPOCHS_LR)

In [137]:
# google-chrome-stable 122 -error memory chrome

In [138]:
# results(for epoch in range(EPOCHS_LR)): to see python_script.py

In [139]:
with torch.no_grad():
    print(model_lr(X_test_tensor[-1:]))

tensor([[0.2142]])


In [140]:
# X_test_tensor[-1:]

In [141]:
torch.save(model_lr, "lr.pth")

In [142]:
# torch.save(model_lr)

In [143]:
model_lr_1 = nn.Sequential(
    nn.Linear(in_features=n_features, out_features=16),
    nn.ReLU(),
    nn.Linear(in_features=16, out_features=32),
    nn.ReLU(),
    nn.Linear(in_features=32, out_features=32),
    nn.ReLU(),
    nn.Linear(in_features=32, out_features=32),
    nn.ReLU(),
    nn.Linear(in_features=32, out_features=32),
    nn.ReLU(),
    nn.Linear(in_features=32, out_features=1))

In [144]:
# torch model_lr_1

In [145]:
print(model_lr_1)

Sequential(
  (0): Linear(in_features=22, out_features=16, bias=True)
  (1): ReLU()
  (2): Linear(in_features=16, out_features=32, bias=True)
  (3): ReLU()
  (4): Linear(in_features=32, out_features=32, bias=True)
  (5): ReLU()
  (6): Linear(in_features=32, out_features=32, bias=True)
  (7): ReLU()
  (8): Linear(in_features=32, out_features=32, bias=True)
  (9): ReLU()
  (10): Linear(in_features=32, out_features=1, bias=True)
)


In [146]:
# torch model_lr_1

In [147]:
metrics_name_2 = torch.nn.MSELoss()

In [148]:
# torch.nn.MSELoss()

In [149]:
Optimizer = optim.SGD(params=model_lr_1.parameters(), lr=0.0001)

In [150]:
# torch.optim model_lr_1

In [151]:
BATCH_SIZE_LR_1 = 16

In [152]:
# torch batch

In [153]:
EPOCHS_LR_1 = 1000

In [154]:
# torch epochs

In [155]:
for epoch in tqdm.trange(EPOCHS_LR_1):
    for i in range((n_data - 1) // BATCH_SIZE_LR_1 + 1):
        start_i = i * BATCH_SIZE_LR_1
        end_i = start_i + BATCH_SIZE_LR_1
        Xb = X_train_tensor[start_i: end_i]
        yb = y_train_tensor[start_i: end_i]
        pred_2 = yb
        loss = loss_func(pred_2, yb + 1)

100%|███████████████████████████████████████| 1000/1000 [06:22<00:00,  2.61it/s]


In [156]:
# for epoch in tqdm.trange(EPOCHS_LR_1)

In [157]:
print(X_test)

[[ 2.14267765  0.039095    0.79884574 ... -0.01879476 -0.09096936
  -0.05191591]
 [ 0.64735879 -0.48534494 -0.66910215 ... -0.20382646 -0.04115568
  -0.14984893]
 [-0.67204021  3.51946916 -0.21742588 ... -0.20382646 -0.20807853
  -0.18840524]
 ...
 [ 0.11959919 -0.67605037 -0.49972355 ... -0.01879476 -0.23577368
  -0.16835596]
 [-0.18826058 -0.48534494 -0.21742588 ... -0.06505269  0.20665959
   0.22106275]
 [ 0.16357915 -0.67605037 -0.49972355 ...  1.8777802  -0.25087444
  -0.15370457]]


In [158]:
# X_test

In [159]:
print(y_test)

360511     2.6
392362     1.4
441510     0.0
106802     0.1
53        12.1
          ... 
326281    16.3
64228      0.6
340331     0.9
1465      18.8
40621      1.1
Name: saturatedfatcontent, Length: 156756, dtype: float64


In [160]:
# y_test

In [161]:
X = torch.randn(BATCH_SIZE_LR_1, 22)

In [162]:
# torch.randn()

In [163]:
y = torch.Tensor([[1.0], [0.0], [0.0], [1.0], [1.0],
    [1.0], [0.0], [0.0], [1.0], [1.0],
    [1.0], [0.0], [0.0], [1.0], [1.0],
    [1.0]])

In [164]:
# torch tensor

In [165]:
for epoch in range(1000):
    y_pred_3 = model_lr_1(X)
    loss = metrics_name_2(y_pred_3, y)
    print('epoch: ', epoch,' loss: ', loss.item())
    Optimizer.zero_grad()
    loss.backward()
    Optimizer.step()

epoch:  0  loss:  0.9682800769805908
epoch:  1  loss:  0.967892050743103
epoch:  2  loss:  0.9675042033195496
epoch:  3  loss:  0.9671165347099304
epoch:  4  loss:  0.9667291045188904
epoch:  5  loss:  0.9663418531417847
epoch:  6  loss:  0.9659549593925476
epoch:  7  loss:  0.9655682444572449
epoch:  8  loss:  0.9651817083358765
epoch:  9  loss:  0.9647954702377319
epoch:  10  loss:  0.9644092917442322
epoch:  11  loss:  0.9640234708786011
epoch:  12  loss:  0.9636378288269043
epoch:  13  loss:  0.9632523655891418
epoch:  14  loss:  0.9628672003746033
epoch:  15  loss:  0.9624821543693542
epoch:  16  loss:  0.9620975255966187
epoch:  17  loss:  0.9617129564285278
epoch:  18  loss:  0.9613285660743713
epoch:  19  loss:  0.9609444737434387
epoch:  20  loss:  0.9605607390403748
epoch:  21  loss:  0.9601771235466003
epoch:  22  loss:  0.9597935080528259
epoch:  23  loss:  0.9594103097915649
epoch:  24  loss:  0.959026575088501
epoch:  25  loss:  0.9586431980133057
epoch:  26  loss:  0.958

In [166]:
# for epoch in range(1000)

In [167]:
def train_1(model_lr_1, loss_fn, optimizer):
    """function_1 train_1"""
    size = len(X_train_tensor)
    model_lr_1.train()
    for batch_size_lr_1 in range(1000):
        y_pred_4 = model_lr_1(X)
        loss = loss_fn(y_pred_4, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if BATCH_SIZE_LR_1 % 16 == 0:
            loss_3 = loss.item(), BATCH_SIZE_LR * len(X)
            print(f'loss: {loss_3}')

In [168]:
# """function_1 train_1"""

In [169]:
def test_1(model_lr_1, loss_fn):
    """function_1 test_1"""
    size = len(X_train_tensor)
    num_batches = size
    model_lr_1.eval()
    test_loss = 0
    with torch.no_grad():
        for batch_size_lr_1 in range(1000):
            y_pred_5 = model_lr_1(X)
            test_loss += loss_fn(y_pred_5, y).item()
            test_loss /= num_batches
            print(f'Test loss: {test_loss}')

In [170]:
# """function_1 test_1"""

In [171]:
with torch.no_grad():
    print(model_lr_1(X_test_tensor[-1:]))

tensor([[-0.0316]])


In [172]:
# X_test_tensor[-1:]

In [173]:
torch.save(model_lr_1, "lr_1.pth")

In [174]:
# torch.save(model_lr_1)